In [ ]:
# If you are using desktop version of geemap, skip this cell
# if you are using google colab code editor, uncomment the last two line of this cell and run it
# Capture is used for suppressing the output.
# %%capture
# %pip install geemap

In [1]:
import ee
import os
import geemap
import ipywidgets as widgets
import ipyleaflet

ee.Authenticate()
ee.Initialize()

#geemap.update_package()
Map=geemap.Map()

Enter verification code: 4/1AZEOvhVS8nGVT-DhU2k3psIjzWnboZeYSfQQXn8M2f7UsE2E6asmLxY2LEc

Successfully saved authorization token.


In [7]:
# Creating year and month slider widget using ipywidgets library
style = {'description_width': 'initial'}

year_slider = widgets.IntRangeSlider(value=[1988, 2020], min=1988 , max=2020,
    step=1,
    description='Year',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='d',
)

month_slider = widgets.IntRangeSlider(value=[1, 12], min=1 , max=12,
    step=1,
    description='Month',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='d',
)

button = widgets.Button(
    description='Update',
    button_style='danger',  # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Click to Update',
#     icon='check',  # (FontAwesome names without the `fa-` prefix)
)


# hbox3 = widgets.HBox([year_slider, month_slider,button])
# hbox3

In [8]:
JRC_yearly=ee.ImageCollection("JRC/GSW1_4/YearlyHistory")
JRC_monthly=ee.ImageCollection("JRC/GSW1_4/MonthlyHistory")
JRC_water=ee.Image("JRC/GSW1_4/GlobalSurfaceWater")

start_year = year_slider.value[0]
end_year = year_slider.value[1]
start_month = month_slider.value[0]
end_month = month_slider.value[1]

do_export = False

visParams_monthly = {'min':0, 'max':2, 'palette':['grey','white','green']}
visParams_trans = {'min':-2, 'max':2, 'palette':['orange','white','blue']}

# filter dates
JRC_monthly = JRC_monthly.filter(ee.Filter.gte('year', start_year)).filter(ee.Filter.lte('year', end_year)).filter(ee.Filter.gte('month', start_month)).filter(ee.Filter.lte('month', end_month))
            
JRC_yearly = JRC_yearly.filter(ee.Filter.gte('year', start_year)).filter(ee.Filter.lte('year', end_year))

# remove zeros?

def func_exf(img):
    return img.updateMask(img)

JRC_monthly = JRC_monthly.map(func_exf)

# https://developers.google.com/earth-engine/guides/image_visualization#colab-python_1

first_month = JRC_monthly.filter(ee.Filter.eq('year', start_year)).mode()
last_month = JRC_monthly.filter(ee.Filter.eq('year', end_year)).mode()

first_month_calc = first_month.unmask(0)
last_month_calc = last_month.unmask(0)


first_month = first_month.updateMask(first_month.neq(1))
last_month = last_month.updateMask(last_month.neq(1))
#Map.add_ee_layer(first_month, visParams_monthly, 'first month (mode)')
#Map.add_ee_layer(last_month, visParams_monthly, 'last month (mode)')

first_month_calc = first_month_calc.remap([0,1,2], [0,1,3])
last_month_calc = last_month_calc.remap([0,1,2], [0,1,3])
transition = last_month_calc.subtract(first_month_calc)
transition = transition.updateMask(transition.neq(0))

Map.add_ee_layer(transition, {min:-3, max:3}, 'transition')
Map.add_ee_layer(transition, visParams_trans, 'transition')

erosion  = transition.eq(2)
sediment = transition.eq(-2)
thesame  = transition.eq(0)
no_data  = transition.eq(-3).Or(transition.eq(-1)).Or(transition.eq(1)).Or(transition.eq(3))

#Map.addLayer(JRC_water, {'bands':'occurrence', 'min':0, 'max':100, 'palette':['white','blue']}, 'JRC water (occ)')
#Map.addLayer(JRC_monthly, visParams_monthly, 'JRC water (monthly)')
#Map.addLayer(JRC_water.select('transition'), {}, 'JRC water (trans)')

year_control = ipyleaflet.WidgetControl(widget=year_slider, position ='bottomright')
# month_control = ipyleaflet.WidgetControl(widget=month_slider, position ='bottomright')
button_control = ipyleaflet.WidgetControl(widget=button, position ='bottomright')

Map.add_control(year_control)
# Map.add_control(month_control)
Map.add_control(button_control)

In [9]:
# Defining a function to calculate erosion-deposition on 'Updated' widget button click 
def update(b):
    
    JRC_yearly=ee.ImageCollection("JRC/GSW1_4/YearlyHistory")
    JRC_monthly=ee.ImageCollection("JRC/GSW1_4/MonthlyHistory")
    JRC_water=ee.Image("JRC/GSW1_4/GlobalSurfaceWater")

    start_year = year_slider.value[0]
    end_year = year_slider.value[1]
    start_month = month_slider.value[0]
    end_month = month_slider.value[1]

    do_export = False

    visParams_monthly = {'min':0, 'max':2, 'palette':['grey','white','green']}
    visParams_trans = {'min':-2, 'max':2, 'palette':['orange','white','blue']}

    # filter dates
    JRC_monthly = JRC_monthly.filter(ee.Filter.gte('year', start_year)).filter(ee.Filter.lte('year', end_year)).filter(ee.Filter.gte('month', start_month)).filter(ee.Filter.lte('month', end_month))

    JRC_yearly = JRC_yearly.filter(ee.Filter.gte('year', start_year)).filter(ee.Filter.lte('year', end_year))

    # remove zeros?

    def func_exf(img):
        return img.updateMask(img)

    JRC_monthly = JRC_monthly.map(func_exf)

    # https://developers.google.com/earth-engine/guides/image_visualization#colab-python_1

    first_month = JRC_monthly.filter(ee.Filter.eq('year', start_year)).mode()
    last_month = JRC_monthly.filter(ee.Filter.eq('year', end_year)).mode()

    first_month_calc = first_month.unmask(0)
    last_month_calc = last_month.unmask(0)


    first_month = first_month.updateMask(first_month.neq(1))
    last_month = last_month.updateMask(last_month.neq(1))
    #Map.add_ee_layer(first_month, visParams_monthly, 'first month (mode)')
    #Map.add_ee_layer(last_month, visParams_monthly, 'last month (mode)')

    first_month_calc = first_month_calc.remap([0,1,2], [0,1,3])
    last_month_calc = last_month_calc.remap([0,1,2], [0,1,3])
    transition = last_month_calc.subtract(first_month_calc)
    transition = transition.updateMask(transition.neq(0))

    Map.add_ee_layer(transition, {min:-3, max:3}, 'transition')
    Map.add_ee_layer(transition, visParams_trans, 'transition')

    erosion  = transition.eq(2)
    sediment = transition.eq(-2)
    thesame  = transition.eq(0)
    no_data  = transition.eq(-3).Or(transition.eq(-1)).Or(transition.eq(1)).Or(transition.eq(3))

# Calling the update defined 'update' function to updated the interactive map based on user input on the slider
button.on_click(update)

# Visualizing the updated intermap
Map

Map(center=[20, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(Togg…